In [ ]:
import pandas as pd

df = pd.read_pickle('shanhua.pkl')

In [ ]:
from selenium import webdriver

driver = webdriver.Chrome()

In [ ]:
def findTarget(html):
    x = str()
    locX = html.find(',')
    for t in html[:locX]:
        if t == '<':
            break
        x += t

    y = str()
    locY = html.find("\n")
    for t in html[locX+1:locY]:
        if t == '<':
            break
        y += t
    
    if (locX == -1 or
        locY == -1):
        return None, None
    
    return x, y

In [ ]:
import time

pd_lat = []
pd_long = []
# pd_door = []
for i in range(df.shape[0]):
    target = df.iloc[i]
    if len(target['address']) == 0:
        pd_lat.append('')
        pd_long.append('')
        continue
    
    address = str()
    address = (target['address'][0]['行政區'] + 
               target['address'][0]['村里鄰'] +
               target['address'][0]['路街段巷弄'] +
               target['address'][0]['號'])
    
    driver.get("http://gps.uhooamber.com/address-to-lat-lng.html")
    input1 =  driver.find_element_by_id("source")
    input1.clear()
    input1.send_keys(address)
    driver.find_element_by_name("B1").click()
    out = driver.find_element_by_id("target")

    timeout = time.time() + 2   # 2 sec from now
    
    try:
        while True:
            c = out.get_attribute('value')
            x, y = findTarget(c)
            if (x != None and
                y != None):
                break
            if time.time() > timeout:
                driver.get("http://gps.uhooamber.com/address-to-lat-lng.html")
                input1 =  driver.find_element_by_id("source")
                input1.clear()
                input1.send_keys(address)
                timeout = time.time() + 2
                
        pd_lat.append(x)
        pd_long.append(y)
        print(i)
        print(x)
        print(y)
        print(address)
        print('==============================')
    except:
        types, message, traceback = sys.exc_info()
        print(types)
        print(message)
        print(address)
        pd_lat.append('')
        pd_long.append('')
        continue